In [1]:
import numpy as np

### PageRank

In [2]:
# Adjacency matrix of links
adjacency_matrix = np.array([
    [0, 1, 1, 1],
    [1, 0, 1, 1],
    [1, 0, 0, 0],
    [1, 0, 1, 0]
])

In [3]:
# Normalize adjacency matrix
col_sums = adjacency_matrix.sum(axis=0)
A = adjacency_matrix / col_sums[np.newaxis, :]
print(A)

[[0.         1.         0.33333333 0.5       ]
 [0.33333333 0.         0.33333333 0.5       ]
 [0.33333333 0.         0.         0.        ]
 [0.33333333 0.         0.33333333 0.        ]]


In [4]:
# Initialize pi
pi = np.matrix([.25,.25,.25,.25]).T

In [5]:
# Use power iteration to update pi
num_iterations = 20
for _ in range(num_iterations):
    numerator = A@pi
    denumerator = np.linalg.norm(numerator)
    pi = numerator/denumerator
    
print('Final pi:', pi)

Final pi: [[0.75526322]
 [0.50350882]
 [0.25175441]
 [0.33567255]]


### Matrix Completion

In [6]:
# Initialize data
X = np.random.rand(4, 5) # values are between 0 and 1
observed_fraction = 0.5
mask = X > observed_fraction
X_obs = X.copy()
X_obs[mask] = -1
print("X:\n", X)
print("X_obs:\n", X_obs)

X:
 [[0.59099394 0.63603308 0.1988718  0.00433864 0.74248003]
 [0.4578318  0.32644027 0.47563335 0.53912158 0.06455628]
 [0.19696818 0.40014478 0.03703266 0.52632507 0.71073452]
 [0.12859307 0.39914934 0.40050152 0.98274261 0.68222808]]
X_obs:
 [[-1.         -1.          0.1988718   0.00433864 -1.        ]
 [ 0.4578318   0.32644027  0.47563335 -1.          0.06455628]
 [ 0.19696818  0.40014478  0.03703266 -1.         -1.        ]
 [ 0.12859307  0.39914934  0.40050152 -1.         -1.        ]]


In [7]:
X_obs.shape

(4, 5)

In [8]:
# Iterative singular value thresholding
def iterative_svt(X_obs, observed_mask, max_iterations, threshold, tol=1e-6):
    X_hat = np.zeros(X_obs.shape)
    X_hat[observed_mask] = X_obs[observed_mask]

    for iteration in range(max_iterations):
        X_old_hat = X_hat.copy()
        U, S, VT = np.linalg.svd(X_hat, full_matrices=False)
        thresholded_S = np.maximum(S - threshold, 0)
    
        X_hat = np.dot(U, np.dot(np.diag(thresholded_S), VT))

        # Update the entries of X corresponding to observed values in M
        X_hat[observed_mask] = X_obs[observed_mask]

        if np.linalg.norm(X_hat - X_old_hat) < tol:
            break

    return X

In [9]:
observed_mask = ~mask
print('mask: \n', mask)
print('observed_mask: \n', observed_mask)

mask: 
 [[ True  True False False  True]
 [False False False  True False]
 [False False False  True  True]
 [False False False  True  True]]
observed_mask: 
 [[False False  True  True False]
 [ True  True  True False  True]
 [ True  True  True False False]
 [ True  True  True False False]]


In [10]:
threshold=1.0
max_iterations=10
recovered_matrix = iterative_svt(X_obs, observed_mask, max_iterations, threshold)

In [11]:
recovered_matrix

array([[0.59099394, 0.63603308, 0.1988718 , 0.00433864, 0.74248003],
       [0.4578318 , 0.32644027, 0.47563335, 0.53912158, 0.06455628],
       [0.19696818, 0.40014478, 0.03703266, 0.52632507, 0.71073452],
       [0.12859307, 0.39914934, 0.40050152, 0.98274261, 0.68222808]])

In [12]:
X

array([[0.59099394, 0.63603308, 0.1988718 , 0.00433864, 0.74248003],
       [0.4578318 , 0.32644027, 0.47563335, 0.53912158, 0.06455628],
       [0.19696818, 0.40014478, 0.03703266, 0.52632507, 0.71073452],
       [0.12859307, 0.39914934, 0.40050152, 0.98274261, 0.68222808]])